### 모델을 만들기 위한 코드입니다. 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from xgboost import plot_importance

path = r"C:/Users/kyung/Desktop/UPT_dev/all_posnet_data/"

data1 = pd.read_csv(path+"all_data_1.csv")
data2 = pd.read_csv(path+"all_data_2.csv")
data3 = pd.read_csv(path+"all_data_3.csv")
data4 = pd.read_csv(path+"all_data_4.csv")

label = pd.read_excel(path+"all_data_label.xlsx", header = None, index_col = 1)

# 첫번째 컬럼빼고 다음에 나오는 컬럼들 삭제
drop_idx1 = []
for i in range(0, len(data1)-4, 4):
    drop_idx1.append(i+3)
    
drop_idx2 = []
for i in range(0, len(data2)-4, 4):
    drop_idx2.append(i+3)
    
drop_idx3 = []
for i in range(0, len(data3)-4, 4):
    drop_idx3.append(i+3)
    
drop_idx4 = []
for i in range(0, len(data4)-4, 4):
    drop_idx4.append(i+3)

data1.drop(data1.index[drop_idx1], inplace = True)
data1.reset_index(drop=True, inplace = True)

data2.drop(data2.index[drop_idx2], inplace = True)
data2.reset_index(drop=True, inplace = True)

data3.drop(data3.index[drop_idx3], inplace = True)
data3.reset_index(drop=True, inplace = True)

data4.drop(data4.index[drop_idx4], inplace = True)
data4.reset_index(drop=True, inplace = True)

# 4개 csv 파일 합치기
frames = [data1, data2, data3, data4]
data = pd.concat(frames, axis = 0)

# 합친 데이터 인덱스 재설정 및 데이터 타입 변환
data.reset_index(drop=True, inplace = True)
data = data.apply(pd.to_numeric)

# posenet score 모을 리스트 생성
score_idx = []
for i in range(0, len(data), 3):
    score_idx.append(i+2)
    
# score점수가 0.5이하면 버리기 -> NaN값으로 처리
columns = data.columns.to_list()
for idx in score_idx:  
    for column in columns:    
        if data.loc[idx][column] < 0.5:
            data.loc[idx-2][column] = np.nan # x  'NaN'
            data.loc[idx-1][column] = np.nan # y
            
# score data만 떼어내기 -> 2346row
score_data = data.loc[score_idx , :]
score_data.reset_index(drop=True, inplace = True)

# x, y 좌표만 남기기
data = data.drop(data.index[score_idx])
data.reset_index(drop=True, inplace = True)

# x좌표 index, y좌표 index 용 리스트 생성
x_idx = []
y_idx = []
for idx in range(0, len(data), 2):
    x_idx.append(idx)
    y_idx.append(idx+1)
    
# x_data, y_data분리
x_data = data.loc[x_idx, :]
x_data.reset_index(drop=True, inplace = True)

y_data = data.loc[y_idx, :]
y_data.reset_index(drop=True, inplace = True)

# 컬럼명 바꾸기용 리스트 생성
x_col = []
y_col = []
for column in columns:
    x_col.append(column+'_x')
    y_col.append(column+'_y')
    
# 컬럼명 바꾸기 
x_data.columns = x_col
y_data.columns = y_col

# 라벨링한 csv파일 전처리 
value = {'classification':label[2]}
label = pd.DataFrame(value, index = label.index)
label.reset_index(drop=True, inplace = True)

# 모두 합치기
all_data = pd.concat([x_data, y_data, label], axis = 1)

# 각 행의 모든 컬럼에서 null값이 20개 이상인 행 제거 => 1419rows
data_20 = all_data.drop(all_data.index[all_data.isnull().sum(axis = 1)>19])
data_20.reset_index(drop=True, inplace = True)

# 모든 특성 학습시키기 
X = data_20[data_20.columns[:34]]
y = data_20[data_20.columns[-1]]

#데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 66)

#모델 학습
model = XGBClassifier(
        learning_rate =0.09,
        n_estimators=110,
        max_depth=7,
        min_child_weight=1,
        gamma=0.3,
        reg_alpha=1e-05,
        subsample=0.61,
        colsample_bytree=0.7,
        seed = 150
) 
model.fit(X_train, y_train)

#평가 
print("훈련 세트 정확도: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(model.score(X_test, y_test)))

ModuleNotFoundError: No module named 'xgboost'

### 만든 모델을 picke로 저장하는 코드입니다. 

In [3]:
import pickle

In [4]:
# 모델 저장하기
pickle.dump(model, open("upt_model.pickle.dat", "wb"))

### pickle로 저장한 모델을 가져오는 코드입니다. 

In [5]:
loaded_model = pickle.load(open("upt_model.pickle.dat", "rb"))

### 예측할 사진의 데이터를 만드는 코드입니다. 

df_data는 data = pd.read_csv(path+"all_data_1.csv")와 같은 형태입니다. 

In [2]:
df_data = [[294.3404010181696, 330.19918906137013, 235.28172759318977, 263.29648096556633, 366.7625854490269, 279.4631815292466, 156.7506582501909, 308.1194027568732, 436.46802523822134, 303.797101686658, 92.80533229896963, 536.0866485227281, 431.1619237212828, 561.3182136950119, 32.890174236336065, 551.97355377842, 451.32081000713754, 553.3488336943044, 62.311236629063956, 528.9984505661056, 439.6857267951582, 551.6956549055379, 115.06657114067309, 546.8307592499424, 397.9426979356608, 522.5189696375272, 59.182472014091445, 524.9163528803128, 407.1228658706849, 524.8728615897041, 67.01323919468962, 544.933471449424, 405.63214384070346, 542.9328548116703, 0.9025035500526428, 0.9798877239227295, 0.9919320940971375, 0.7146050333976746, 0.3866192698478699, 0.08203981816768646, 0.0007519477512687445, 0.004390000365674496, 0.0002444644342176616, 0.010003565810620785, 0.0003872329252772033, 0.009263763204216957, 0.0014169629430398345, 0.00369338714517653, 0.000635435339063406, 0.005955243483185768, 0.0014176637632772326], [295.0202813868072, 327.97099502993296, 230.83198306162353, 261.5198988310048, 365.63287419908244, 275.9570716612296, 157.46961489625625, 300.625376826081, 437.79396488489044, 299.37039486599156, 76.2910127697337, 539.3873893998758, 430.0054648570132, 557.4070836456729, 32.369889441630335, 552.187340696093, 452.21396495734666, 554.2426333820796, 66.376087656923, 529.8692840685546, 440.33822423831316, 552.0061233873578, 92.38880160924657, 545.0566022208998, 402.4472881908148, 540.2279176462583, 50.77030409845554, 527.7602983432516, 406.0270681045425, 524.0798994329134, 83.32372379111098, 543.7749891453825, 404.868937168083, 542.7586340568434, 0.9530836939811707, 0.9605786204338074, 0.9809536933898926, 0.7411233186721802, 0.7245631217956543, 0.006078552920371294, 0.000936977390665561, 0.0028391049709171057, 0.00026460198569111526, 0.010042977519333363, 0.00037792796501889825, 0.005841980222612619, 0.0006847854820080101, 0.004518385976552963, 0.0004703481390606612, 0.007715870160609484, 0.0012383057037368417], [294.2200112448372, 333.7277142814467, 236.4902870093793, 265.3624498148559, 369.31122534807537, 281.0117143020783, 159.29253984600967, 307.6884532598422, 437.23882051157045, 296.90577269080177, 94.09883473816535, 536.9656601182411, 400.3437581973776, 544.0386597539338, 33.141182083958824, 552.9368040787142, 411.1077209411253, 543.9665023949544, 82.1192953006123, 530.3673267844217, 402.101940270161, 522.0103350203761, 116.50507538227492, 547.5001069381683, 383.630241240534, 540.8037437039841, 83.67252336037711, 538.1357396392515, 378.4864788861342, 519.0974797761176, 82.7104741716289, 543.615329097694, 368.06208740153784, 544.7026159681784, 0.9675635099411011, 0.8970657587051392, 0.9885512590408325, 0.6728065609931946, 0.46137765049934387, 0.05227629467844963, 0.00019077674369327724, 0.0025784792378544807, 0.00013077988114673644, 0.00934219267219305, 0.0001955316256498918, 0.006251676008105278, 0.001501288264989853, 0.0047569978050887585, 0.00025950471172109246, 0.008261336013674736, 0.0028665042482316494], [295.25156717281226, 337.4900635099507, 230.53547429942512, 268.9073908017195, 365.8338652347895, 283.4302161541023, 161.20795729270645, 313.0417081191986, 439.0955138791735, 300.42833898149024, 88.31444271709364, 544.0678419001865, 446.8800130801901, 562.3084304318342, 48.65460698609388, 547.1803720808125, 470.19623679270444, 553.9804640910035, 80.64388591374916, 530.6564622241966, 458.9521886414925, 553.5526384050457, 93.69715546314387, 545.113283000002, 420.21649325733455, 540.9228497587699, 48.03300686381476, 526.5449896184971, 405.4645243962286, 519.2406368447502, 83.63314774050798, 544.405256094827, 404.9289763391138, 543.5772354693959, 0.9772679209709167, 0.9503750205039978, 0.9908140301704407, 0.5333714485168457, 0.6805264353752136, 0.007035363465547562, 0.000699736934620887, 0.004074904136359692, 0.0002335568715352565, 0.008551842533051968, 0.00023597879044245929, 0.0066926442086696625, 0.000967338855843991, 0.006002673879265785, 0.0002514734223950654, 0.010053850710391998, 0.0020748174283653498], [295.24907457132934, 333.4467151994916, 232.16224793673047, 266.8160083548164, 367.24193739555255, 282.0111363946312, 162.79736491612027, 309.02964670653313, 441.40141346660903, 299.2442264883091, 87.66740021427398, 542.8437405668753, 449.48445580710825, 563.5637989466338, 49.126852499887036, 545.8973466270646, 470.6155320825711, 553.3433033427002, 78.79134836906883, 528.670143937201, 461.25480208214617, 557.417530121218, 94.51790782383512, 543.9855313636887, 422.5503770455988, 541.3148795574963, 62.9468206486232, 523.0602458209339, 423.93937029924916, 542.8209364294286, 84.7970964980558, 542.9839790467045, 423.17333500246167, 546.7197296844881, 0.9624528288841248, 0.9565593004226685, 0.9943510293960571, 0.48014265298843384, 0.4679611325263977, 0.006585065275430679, 0.000434998917626217, 0.0037023837212473154, 0.00019238250388298184, 0.008411413989961147, 0.0001952418388100341, 0.006910066585987806, 0.0006390096968971193, 0.004554911982268095, 0.0009599465993233025, 0.009082612581551075, 0.0009625295642763376], [296.64012983387386, 333.3985210664315, 233.45939701088002, 266.5530302634901, 367.05214905283106, 281.4352686016612, 159.61077597635375, 305.00860732326083, 441.0673881737999, 301.53165707885614, 92.06439758546401, 534.0735927673892, 430.7530357621805, 555.2677845331265, 29.304419634567694, 550.0305866091784, 450.0908234287316, 551.1250116455723, 69.70283218168879, 525.1833818807928, 438.03645874269057, 550.7387453401592, 113.92435140628936, 547.2454530132608, 399.7543500520334, 539.5170812395736, 61.54580131239095,
523.280246972558, 379.7877354458784, 522.7146607289611, 85.34614061781576, 542.7986469307176, 367.98792351275625, 547.1866382679468, 0.9660137295722961, 0.9517143368721008, 0.9918251633644104, 0.514086902141571, 0.5132740139961243, 0.09093323349952698, 0.0006176643655635417, 0.004274587146937847, 0.0001586378930369392, 0.006844040006399155, 0.00023844174575060606, 0.009384626522660255, 0.0013076375471428037, 0.004913578741252422, 0.0002999073767568916, 0.01030948106199503, 0.003018633695319295], [274.143659714482, 336.9772157918519, 217.9044898664448, 268.1455142061475, 356.62479866534414, 278.2924533400017, 145.75881502853798, 294.3124986030686, 432.9573958030409, 318.0917694535054, 65.98859462315886, 548.1810448635032, 455.4307437336421, 561.5971743700729, 30.559502047072556, 553.7485387483592, 470.8399278114979, 554.3701262061504, 79.78997821155451, 528.0874950305316, 460.4355740652718, 556.95816957255, 92.52332955634813, 545.0238898486439, 440.2425566966865, 543.7581430738361, 63.42035275925582, 525.7780426224951, 445.5131838316889, 548.1480217795496, 84.15527118859404, 541.7457726399903, 442.3977356379181, 550.3544970057619, 0.9867554903030396, 0.9722554087638855, 0.9867628216743469, 0.4391316771507263, 0.9240501523017883, 0.005000313278287649, 0.0003994677681475878, 0.003375524654984474, 0.00015281194646377116, 0.0064096772111952305, 0.00015743420226499438, 0.00797896645963192, 0.0006378249963745475, 0.0035524494014680386, 0.0008417664212174714, 0.006009665317833424, 0.0005933562642894685], [259.6230359681895, 337.6120455548077, 203.2798069161429, 268.24621760869167, 339.69524799554165, 276.9598029269059, 140.4084989060339, 280.34557562238973, 430.8479802766796, 314.40395224742, 64.0276782766914, 542.8179758175518, 467.02072812853686, 565.3295486265985, 30.00014442889028, 550.2501986156286, 489.6431261442557, 555.9168326303032, 48.2178565779443, 533.6598491284929, 479.7710913345368, 560.2697760046607, 92.94799804303733, 541.156082421961, 438.9985804644149, 546.7245836373066, 48.087337516922844, 525.956954130944, 424.1261828650891, 545.0017002266896, 65.45456655644318, 545.0714530599429, 422.9646985017558, 546.3511713553722, 0.9528825283050537, 0.992141604423523, 0.9842488169670105, 0.4151732921600342, 0.876139760017395, 0.01233613584190607, 0.0009294277406297624, 0.005606833845376968, 0.00023252870596479625, 0.005990162491798401, 0.00018030060164164752, 0.00810507033020258, 0.0007199745159596205, 0.0033445563167333603, 0.0010886871023103595, 0.004501515533775091, 0.0008054896024987102], [252.5136723422429, 342.0360089307819, 193.36748600162014, 270.16749353236116, 335.0115582794251, 281.2539865313401, 133.7832912410529, 300.35192146147756, 426.2644676672861, 329.5100797350018, 74.27424372897542, 544.8845116905043, 462.00459272664796, 563.5260599239971, 9.174431983134241, 554.2482322370502, 491.75545622981053, 554.955182372924, 21.197492789454714, 550.8367927981094, 478.53286535543214, 555.7781933299012, 93.76621952095263, 541.1645394215881, 438.8975063377703, 546.6943641543628, 47.97936700095602, 524.5951322482625, 424.1649807396549, 547.5697136980666, 65.72737516867562, 546.129779700542, 424.88551451022954, 548.7888576938832, 0.9745827913284302, 0.9003944993019104, 0.9740741848945618, 0.13667427003383636, 0.8958275318145752, 0.03143520653247833, 0.0005619071307592094, 0.001890619401820004, 0.00023132836213335395, 0.0013228723546490073, 0.00016153840988408774, 0.00719345873221755, 0.0006950747920200229, 0.0020804887171834707, 0.0008890141034498811, 0.0032327850349247456, 0.0005791608127765357], [245.53093503034813, 346.0082105947452, 187.39184244873542, 275.71178787431006, 327.34614210685254, 285.5861811570718, 133.72895857313995, 295.84621989751, 423.9443259997147, 334.16833167584605, 79.05379843759829, 540.0051297316369, 440.059101803202, 555.2167454955563, 8.557922409333969, 554.2020782378597, 469.65044687067723, 554.9683039337096, 22.780222629877244, 549.3057708025458, 459.4334501193563, 557.2452355198697, 91.78187185705792, 543.0893844282123, 403.2102032743949, 542.0824781989667, 44.669733350665524, 524.4142160089442, 377.99927200685806, 523.15912150761, 64.76772590038524, 543.1189111061019, 366.6546853432953, 545.2326265139359, 0.9626593589782715, 0.9243665337562561, 0.9940714240074158, 0.16438019275665283, 0.9510586261749268, 0.024815138429403305, 0.00221130158752203, 0.001382472924888134, 0.00018187082605436444, 0.0013340434525161982, 0.00018111633835360408, 0.0060203224420547485, 0.001374591258354485, 0.002372384537011385, 0.0006615885067731142, 0.004285971634089947, 0.004410055000334978]]

In [6]:
def made(result_list):
    result_list_2 = []
    for i in result_list:
        a_xy = i[:34]
        score_list = i[34:]
        x_list = []
        y_list = []


        for j in range(len(a_xy)):
            if j == 0:
                x_list.append(a_xy[j])
            elif j % 2 == 0 :
                x_list.append(a_xy[j])
            else:
                y_list.append(a_xy[j])
        result_list_2.append([x_list,y_list,score_list])

    return result_list_2

In [31]:
def made_dataframe(input_):
    key= ["nose","leftEye","rightEye","leftEar","rightEar","leftShoulder","rightShoulder","leftElbow","rightElbow","leftWrist","rightWrist","leftHip","rightHip","leftKnee","rightKnee","leftAnkle","rightAnkle"]
    input_data = pd.DataFrame(columns = key)
    idx = 0
    for i in range(len(input_)):
        x_ = input_[i][0]
        y_ = input_[i][1]
        score_ = input_[i][2]

        for j in range(4):
            if j == 0:
                input_data.loc[j + idx] =x_
            elif j == 1:
                input_data.loc[j + idx] =y_
            elif j == 2:
                input_data.loc[j + idx] =score_
            else:
                input_data.loc[j + idx] =key
        idx += 4
    return input_data

In [49]:
def make_data(df_data):
#     첫번째 컬럼빼고 다음에 나오는 부위컬럼들 삭제
    drop_idx = []
    for i in range(0, len(df_data)-4, 4):
        drop_idx.append(i+3)
    
    df_data.drop(df_data.index[drop_idx], inplace = True)
    df_data.reset_index(drop=True, inplace = True)
    
    
    # 모든 값 str -> float형태로 
    df_data = df_data.apply(pd.to_numeric)
    
    # posenet score 모을 리스트 생성
    score_idx = []
    for i in range(0, len(df_data), 3):
        score_idx.append(i+2)
        
    # score점수가 0.5이하면 버리기 -> NaN값으로 처리
    columns = df_data.columns.to_list()
    for idx in score_idx:  
        for column in columns:    
            if df_data.loc[idx][column] < 0.5:
                df_data.loc[idx-2][column] = np.nan # x  'NaN'
                df_data.loc[idx-1][column] = np.nan # y
                
    # x, y 좌표만 남기기
    df_data = df_data.drop(df_data.index[score_idx])
    df_data.reset_index(drop=True, inplace = True)
    
    # x좌표 index, y좌표 index 용 리스트 생성
    x_idx = []
    y_idx = []
    for idx in range(0, len(df_data), 2):
        x_idx.append(idx)
        y_idx.append(idx+1)
        
    # x_data, y_data분리
    x_data = df_data.loc[x_idx, :]
    x_data.reset_index(drop=True, inplace = True)

    y_data = df_data.loc[y_idx, :]
    y_data.reset_index(drop=True, inplace = True)
    
    # 컬럼명 바꾸기용 리스트 생성
    x_col = []
    y_col = []
    for column in columns:
        x_col.append(column+'_x')
        y_col.append(column+'_y')
        
    # 컬럼명 바꾸기 
    x_data.columns = x_col
    y_data.columns = y_col
    
    # 모두 합치기
    all_data = pd.concat([x_data, y_data], axis = 1)
    all_data = all_data.apply(pd.to_numeric)
    
    return all_data #이 결과를 

In [50]:
df = made(df_data)

In [51]:
df2 = made_dataframe(df)

In [56]:
df2.iloc[:-1,:]

,nose,leftEye,rightEye,leftEar,rightEar,leftShoulder,rightShoulder,leftElbow,rightElbow,leftWrist,rightWrist,leftHip,rightHip,leftKnee,rightKnee
0,294.34,235.282,366.763,156.751,436.468,92.8053,431.162,32.8902,451.321,62.3112,439.686,115.067,397.943,59.1825,407.123
1,330.199,263.296,279.463,308.119,303.797,536.087,561.318,551.974,553.349,528.998,551.696,546.831,522.519,524.916,524.873
2,0.902504,0.979888,0.991932,0.714605,0.386619,0.0820398,0.000751948,0.00439,0.000244464,0.0100036,0.000387233,0.00926376,0.00141696,0.00369339,0.000635435
3,295.02,230.832,365.633,157.47,437.794,76.291,430.005,32.3699,452.214,66.3761,440.338,92.3888,402.447,50.7703,406.027
4,327.971,261.52,275.957,300.625,299.37,539.387,557.407,552.187,554.243,529.869,552.006,545.057,540.228,527.76,524.08
5,0.953084,0.960579,0.980954,0.741123,0.724563,0.00607855,0.000936977,0.0028391,0.000264602,0.010043,0.000377928,0.00584198,0.000684785,0.00451839,0.000470348
6,294.22,236.49,369.311,159.293,437.239,94.0988,400.344,33.1412,411.108,82.1193,402.102,116.505,383.63,83.6725,378.486
7,333.728,265.362,281.012,307.688,296.906,536.966,544.039,552.937,543.967,530.367,522.01,547.5,540.804,538.136,519.097
8,0.967564,0.897066,0.988551,0.672807,0.461378,0.0522763,0.000190777,0.00257848,0.00013078,0.00934219,0.000195532,0.00625168,0.00150129,0.004757,0.000259505
9,295.252,230.535,365.834,161.208,439.096,88.3144,446.88,48.6546,470.196,80.6439,458.952,93.6972,420.216,48.033,405.465


In [53]:
make_data(df2)

KeyError: ('nose', 'leftEye', 'rightEye', 'leftEar', 'rightEar', 'leftShoulder', 'rightShoulder', 'leftElbow', 'rightElbow', 'leftWrist', 'rightWrist', 'leftHip', 'rightHip', 'leftKnee', 'rightKnee', 'leftAnkle', 'rightAnkle')

---
이 아래부터는 예시입니다. 
### make_data함수가 제대로 작동하는지 확인하기위한 예시 코드입니다.

In [10]:
# 처리하기 전 단계인 csv 
test_data = pd.read_csv(path+"all_data_1.csv")
test_data

,nose,leftEye,rightEye,leftEar,rightEar,leftShoulder,rightShoulder,leftElbow,rightElbow,leftWrist,rightWrist,leftHip,rightHip,leftKnee,rightKnee,leftAnkle,rightAnkle
0,0,0.007,0.008,0.033,0.056,0.146,0.16,0.191,0.509,0.174,0.965,0.265,0.458,0.181,0.158,0.153,1
1,0,0.048,0.174,0.492,0.334,0.195,0.211,0.824,0.425,1.467,0.943,0.354,0.886,0.987,0.97,1.344,1
2,0.941,0.271,0.966,0.104,0.964,0.469,0.968,0.928,0.855,0.774,0.82,0.057,0.08,0.125,0.133,0.067,0.096
3,nose,leftEye,rightEye,leftEar,rightEar,leftShoulder,rightShoulder,leftElbow,rightElbow,leftWrist,rightWrist,leftHip,rightHip,leftKnee,rightKnee,leftAnkle,rightAnkle
4,0,0.019,0.07,0.255,0.374,0.821,0.733,1.004,0.913,4.399,0.88,1.33,1.168,0.991,0.949,1.018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1762,0.855,0.63,0.725,0.251,0.729,0.641,0.961,0.55,0.729,0.708,0.808,0.168,0.218,0.194,0.186,0.097,0.078
1763,nose,leftEye,rightEye,leftEar,rightEar,leftShoulder,rightShoulder,leftElbow,rightElbow,leftWrist,rightWrist,leftHip,rightHip,leftKnee,rightKnee,leftAnkle,rightAnkle
1764,0,0.023,0.06,0.229,0.115,1.124,0.67,3.579,1.068,6.685,1.005,0.137,0.108,3.396,1.216,0.949,1
1765,0,0.092,0.036,0.228,0.125,0.148,0.023,0.225,0.521,0.59,1.025,1.238,1.22,0.2,0.51,1.028,1


### DataFrame으로 불러온 csv파일을 make_data함수에 넣으면

In [11]:
result = make_data(test_data)
result

,nose_x,leftEye_x,rightEye_x,leftEar_x,rightEar_x,leftShoulder_x,rightShoulder_x,leftElbow_x,rightElbow_x,leftWrist_x,...,leftElbow_y,rightElbow_y,leftWrist_y,rightWrist_y,leftHip_y,rightHip_y,leftKnee_y,rightKnee_y,leftAnkle_y,rightAnkle_y
0,0.0,NaN,0.008,NaN,0.056,NaN,0.160,0.191,0.509,0.174,...,0.824,0.425,1.467,0.943,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.019,NaN,0.255,NaN,0.821,0.733,1.004,NaN,NaN,...,0.520,NaN,NaN,NaN,NaN,NaN,0.545,0.469,1.053,1.0
2,0.0,NaN,NaN,0.226,NaN,2.048,2.019,NaN,1.618,NaN,...,NaN,0.514,NaN,1.028,0.124,NaN,0.526,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.033,NaN,0.116,NaN,0.256,0.207,0.975,NaN,NaN,...,0.274,NaN,NaN,NaN,NaN,NaN,0.641,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,0.0,0.148,0.092,NaN,0.366,0.511,1.137,1.583,1.289,1.519,...,0.501,0.514,0.947,0.886,NaN,NaN,NaN,NaN,NaN,NaN
438,0.0,0.078,0.087,0.000,NaN,1.174,0.809,1.241,1.086,NaN,...,0.418,0.475,NaN,NaN,NaN,NaN,0.121,0.395,0.998,1.0
439,0.0,NaN,0.001,NaN,0.004,0.124,0.201,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,0.0,0.005,0.001,NaN,0.055,0.124,0.165,0.186,0.207,0.989,...,1.151,0.959,1.014,1.028,NaN,NaN,NaN,NaN,NaN,NaN


### ↑ 생성한 모델에 넣을 수 있는 형태로 처리된 결과입니다. 

### pickle파일로 저장해놓은 모델을 불러오면

In [12]:
import pickle

loaded_model = pickle.load(open("upt_model.pickle.dat", "rb"))

### 따로 변수로 만들어놓은 result를 넣어도되고, make_data(test_data)를 넣어도됩니다.

In [13]:
y_pred = loaded_model.predict(result) 
predictions = [round(value) for value in y_pred]
print(predictions) 

[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 예시 데이터의 결과는 리스트로 1,0값이 출력됩니다. 